In [ ]:
#This program using to estimate our program
#Using trial restaurant.xml only

In [1]:
#Part 1: XML to CSV
import xml.etree.ElementTree as ET
import xml.dom.minidom
from time import time
import csv
                ################################################
                ########### 1. Resturant Trial Data ############
                ################################################
start=time()
doc = xml.dom.minidom.parse('/Users/xizhaohan/Desktop/498project2/data/trial/restaurants-trial.xml')
docroot = doc.documentElement

text=docroot.getElementsByTagName('text')
Sentences=[]
for sent in text:
    Sentences.append(sent.firstChild.data)
    
    
    
tree = ET.ElementTree(file='/Users/xizhaohan/Desktop/498project2/data/trial/restaurants-trial.xml')
treeroot=tree.getroot()

Term_and_Polarity=[]
Category_and_Polarity=[]
for sentence in treeroot:
    tempsent1=''
    tempsent2=''
    if len(sentence)==3:
        for AT in sentence[1]:
            Term=AT.attrib
            tempsent1+='('+Term['term']+'#'+Term['polarity']+')'
        for AC in sentence[2]:
            Category=AC.attrib
            tempsent2+='('+Category['category']+'#'+Category['polarity']+')'
    if len(sentence)==2:
        for A in sentence[1]:
            Aspect=A.attrib
            if 'term' in Aspect:
                tempsent1+='('+Aspect['term']+'#'+Aspect['polarity']+')'
            else:
                tempsent2+='('+Aspect['category']+'#'+Aspect['polarity']+')'

    Term_and_Polarity.append(tempsent1)
    Category_and_Polarity.append(tempsent2)

with open( '/Users/xizhaohan/Desktop/498estimate/Trial_restaurant_origianlContent.csv', 'w') as file:
    writer = csv.writer(file)
    for col1,col2,col3 in zip(Term_and_Polarity,Category_and_Polarity,Sentences):
        writer.writerow([str(col1),str(col2),str(col3)])
file.close()

stop=time()
print('total time is:',stop-start,'seconds')

total time is: 0.015631914138793945 seconds


In [1]:
#Part 2 Get outcomes of subtask2

import csv
from time import time
from textblob import TextBlob
from nltk.corpus import wordnet as wn
from nltk.parse.stanford import StanfordDependencyParser
path_to_jar = '/Users/xizhaohan/Desktop/Text Mining/project1/My work/stanford-parser-full-2014-08-27/stanford-parser.jar'
path_to_models_jar = '/Users/xizhaohan/Desktop/Text Mining/project1/My work/stanford-parser-full-2014-08-27/stanford-parser-3.4.1-models.jar'
dependency_parser = StanfordDependencyParser(path_to_jar=path_to_jar, path_to_models_jar=path_to_models_jar)

                    #############################################
                    ############  I. Read CSV files  ############
                    #############################################
start=time()
###########  I-1:read Trial_restaurant_origianlContent.csv ###########
#I-1-1: get lists
Sent=[]
AT=[]
AC=[]
SentsNum=0
with open('/Users/xizhaohan/Desktop/498estimate/Trial_restaurant_origianlContent.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
    # row is a list:
    #['(AT1#ATP1)(AT2#ATP2)', '(AC1#ACP1)(AC2#ACP2)', 'sent']
        Sent.append(row[2])
        ATraw=row[0].strip('()').split(')(')
        ACraw=row[1].strip('()').split(')(')
        samesentAT=[tuple(ele1.split('#')) for ele1 in ATraw]
        samesentAC=[tuple(ele2.split('#')) for ele2 in ACraw]
        AT.append(samesentAT)
        AC.append(samesentAC)
        SentsNum+=1
file.close()

#I-1-2: get total dictionary
All_dict={}
for ID in range(1,SentsNum+1):
    #ID in dict form 1 to 100(total number of trial sentences)
    All_dict[str(ID)]={'sentence':Sent[ID-1], 'AspectTerm':AT[ID-1], 'AspectCategroy':AC[ID-1]}
    #TestAll_dict is a 2-D dictionary:
    #{'ID1':{'sentence':'sent','AspectTerm':[(AT1,ATP1),(AT2,ATP2)],'AspectCategory':[(AC1,ACP1),(AC2,ACP2)]}, 'ID2':{}, ...}

###########  I-2:read 4 CSV files with deleted sentiment adj ###########
SentimentDict_Deleted={'positive':[],'negative':[], 'neutral':[] , 'conflict':[]}
with open('/Users/xizhaohan/Desktop/Subtask2_4/positive_deleted_RST.csv', 'r') as file1:
    reader1 = csv.reader(file1)
    for row in reader1:
        SentimentDict_Deleted['positive']+=row
file1.close()
with open('/Users/xizhaohan/Desktop/Subtask2_4/negative_deleted_RST.csv', 'r') as file2:
    reader2 = csv.reader(file2)
    for row in reader2:
        SentimentDict_Deleted['negative']+=row
file2.close()
with open('/Users/xizhaohan/Desktop/Subtask2_4/neutral_deleted_RST.csv', 'r') as file3:
    reader3 = csv.reader(file3)
    for row in reader3:
        SentimentDict_Deleted['neutral']+=row
file3.close()
with open('/Users/xizhaohan/Desktop/Subtask2_4/conflict_deleted_RST.csv', 'r') as file4:
    reader4 = csv.reader(file4)
    for row in reader4:
        SentimentDict_Deleted['conflict']+=row
file4.close()


                    ########################################################
                    ############  II. Extend 4 sentiment lists  ############
                    ########################################################

SentimentDict_Extended={'positive':[],'negative':[], 'neutral':[] , 'conflict':[]}
for adj in SentimentDict_Deleted['positive']:
    allsyn=wn.synsets(adj)
    for word in allsyn:
        SentimentDict_Extended['positive'].append(str(word).strip('Synset()').split('.')[0].strip('\''))
SentimentDict_Extended['positive']=list(set(SentimentDict_Extended['positive']))

for adj in SentimentDict_Deleted['negative']:
    allsyn=wn.synsets(adj)
    for word in allsyn:
        SentimentDict_Extended['negative'].append(str(word).strip('Synset()').split('.')[0].strip('\''))
SentimentDict_Extended['negative']=list(set(SentimentDict_Extended['negative']))

SentimentDict_Extended['neutral']=SentimentDict_Deleted['neutral']
SentimentDict_Extended['conflict']=SentimentDict_Deleted['conflict']

                    ###########################################################
                    ############  III. Judge polarity of trial AT  ############
                    ###########################################################
            
estimate_subtask2={}
for ID in range(1,SentsNum+1):
    estimate_subtask2[str(ID)]={'sentence':Sent[ID-1], 'AspectTerm':[], 'AspectCategroy':[]}
    for p in AT[ID-1]:
        estimate_subtask2[str(ID)]['AspectTerm'].append(p[0])
    for p in AC[ID-1]:
        estimate_subtask2[str(ID)][ 'AspectCategroy'].append(p[0])
    #estimate_subtask2 is a 2-D dictionary:
    #{'ID1':{'sentence':'sent','AspectTerm':[AT1,AT2],'AspectCategory':[AC1,AC2]}, 'ID2':{}, ...}

OutDict_subtask2={}
for ID in range(1,SentsNum+1):
    #if ID>30:
        #break
    #III-1: drag element
    currentSent=estimate_subtask2[str(ID)]['sentence']
    currentAT=estimate_subtask2[str(ID)]['AspectTerm']
    currentAC=estimate_subtask2[str(ID)]['AspectCategroy']
    OutDict_subtask2[str(ID)]={'sentence':currentSent, 'AspectTerm':[], 'AspectCategroy':currentAC}
    if len(currentAT[0])==0:
        continue
    #III-2 parse each sentence and get all (N,Adj) pairs in list form
    Ntag=('NN','NNS','NNP','NNPS')
    ADJtag=('JJ','JJR','JJS')
    N_Adj_pair=[]
    try:
        ParseSent=list(dependency_parser.raw_parse(currentSent).__next__().triples())
    except BaseException:  continue
    #ParseSent looks like:
    #[(('horrible', 'JJ'), 'cc', ('But', 'CC')), 
    #(('horrible', 'JJ'), 'nsubj', ('staff', 'NN')), 
    #(('staff', 'NN'), 'det', ('the', 'DT')), 
    #(('to', 'TO'), 'pobj', ('us', 'PRP'))]
    for outpair in ParseSent:
        if outpair[0][1] in Ntag and outpair[2][1] in ADJtag:
            N_Adj_pair.append(tuple((outpair[0][0], outpair[2][0])))
        elif outpair[2][1] in Ntag and outpair[0][1] in ADJtag:
            N_Adj_pair.append(tuple((outpair[2][0], outpair[0][0])))
    #N_Adj_pair looks like:
    #[('food', 'fair'), ('factor', 'only'), ('deficiencies', 'other')]
    
    #III-3: detect whether N is AT
    AT_ADJ=[]
    for pair in N_Adj_pair:
        noun=pair[0]
        adj=pair[1]
        for at in currentAT:
            if noun==at or noun in at:
                AT_ADJ.append(tuple((at, adj)))
    #AT_ADJ looks like:
    #[('sushi', 'best'), ('place', 'clean')]
    #its noun is real AT, its adj is candidate
    #III-4: detect polarity of AT in AT_ADJ by adj
    AspectTerm=[]
    for p in AT_ADJ:
        if p[1] in SentimentDict_Extended['positive']:
            AspectTerm.append(tuple((p[0], 'positive')))
        elif p[1] in SentimentDict_Extended['negative']:
            AspectTerm.append(tuple((p[0], 'negative')))
        elif p[1] in SentimentDict_Extended['conflict']:
            AspectTerm.append(tuple((p[0], 'conflict')))
        else:
            AspectTerm.append(tuple((p[0], 'neutral')))
    if len(AT_ADJ)!=0:
        OutDict_subtask2[str(ID)]['AspectTerm']=AspectTerm
        #OutDict_subtask2 stores the outcome of this program in dictionary form,looks like:
        #{'id1':{'sentence':..., 'AspectTerm':[(AT1,ATP1),(AT2,ATP2)], 'AspectCategory':[AC1,AC2]}, 'id2':{}}
    else:
        blob = TextBlob(currentSent)
        tempATP=blob.sentiment.polarity
        if tempATP>0:
            for term in currentAT:
                AspectTerm.append(tuple((term, 'positive')))
        elif tempATP<0:
            for term in currentAT:
                AspectTerm.append(tuple((term, 'negative')))
        else:
            for term in currentAT:
                AspectTerm.append(tuple((term, 'neutral')))
        OutDict_subtask2[str(ID)]['AspectTerm']=AspectTerm
        
    OutDict_subtask2[str(ID)]['AspectTerm']=list(set(OutDict_subtask2[str(ID)]['AspectTerm']))
    OutDict_subtask2[str(ID)]['AspectCategroy']=list(set(OutDict_subtask2[str(ID)]['AspectCategroy']))


                    ##############################################
                    ############  IV. Write into CSV  ############
                    ##############################################               
AT_Write=[]
AC_Write=[]
Sent_Write=[]
for ID in range(1,SentsNum+1):
    #if ID>30:
        #break
    Sent_Write.append(OutDict_subtask2[str(ID)]['sentence'])
    sameSentAT=''
    sameSentAC=''
    for ele1 in OutDict_subtask2[str(ID)]['AspectTerm']:
        sameSentAT+='('+ele1[0]+'#'+ele1[1]+')'
    for ele2 in OutDict_subtask2[str(ID)]['AspectCategroy']:
        sameSentAC+='('+ele2+')'
    AT_Write.append(sameSentAT)
    AC_Write.append(sameSentAC) 
    
with open( '/Users/xizhaohan/Desktop/498estimate/Subtask2estimate_restaurant.csv', 'w') as file:
    writer = csv.writer(file)
    for col1,col2,col3 in zip(AT_Write,AC_Write,Sent_Write):
        writer.writerow([str(col1),str(col2),str(col3)])
file.close()

stop=time()
print('Total time is:', stop-start,'seconds')



Total time is: 51.47542691230774 seconds


In [2]:
#Part 3 Get outcomes of subtask4

import csv
from time import time
from textblob import TextBlob
from nltk.corpus import wordnet as wn
from nltk.parse.stanford import StanfordDependencyParser
path_to_jar = '/Users/xizhaohan/Desktop/Text Mining/project1/My work/stanford-parser-full-2014-08-27/stanford-parser.jar'
path_to_models_jar = '/Users/xizhaohan/Desktop/Text Mining/project1/My work/stanford-parser-full-2014-08-27/stanford-parser-3.4.1-models.jar'
dependency_parser = StanfordDependencyParser(path_to_jar=path_to_jar, path_to_models_jar=path_to_models_jar)

start=time()
                    #############################################
                    ############  I. Read CSV files  ############
                    #############################################   

#I-1: read Subtask2estimate_restaurant.csv
#I-1-1: get lists
task2Sent=[]
task2AT=[]
task2AC=[]
task2SentsNum=0
with open('/Users/xizhaohan/Desktop/498estimate/Subtask2estimate_restaurant.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
    # row is a list:
    #['(AT1#ATP1)(AT2#ATP2)', '(AC1)(AC2)', 'sent']
        task2Sent.append(row[2])
        ATraw=row[0].strip('()').split(')(')
        ACraw=row[1].strip('()').split(')(')
        samesentAT=[tuple(ele1.split('#')) for ele1 in ATraw]
        samesentAC=[ele2 for ele2 in ACraw]
        task2AT.append(samesentAT)
        task2AC.append(samesentAC)
        task2SentsNum+=1
file.close()

#I-1-2: get total dictionary
Task2All_dict={}
for ID in range(1,task2SentsNum+1):
    #ID in dict form 1 to 100(total number of trial sentences)
    Task2All_dict[str(ID)]={'sentence':task2Sent[ID-1], 'AspectTerm':task2AT[ID-1], 'AspectCategroy':task2AC[ID-1]}
    #Task2All_dict is a 2-D dictionary:
    #{'ID1':{'sentence':'sent','AspectTerm':[(AT1,ATP1),(AT2,ATP2)],'AspectCategory':[AC1,AC2]}, 'ID2':{}, ...}

#I-2: read AT-AC.csv
#I-2-1: get lists
AT2AC_AT=[]
AT2AC_AC=[]
AT2AC_Num=0
Label_AC={'1':'food', '2':'service', '3':'price', '4':'ambience', '5':'anecdotes/miscellaneous', '':'anecdotes/miscellaneous'}
with open('/Users/xizhaohan/Desktop/Subtask2_4/AT_AC.csv', 'r') as file_AT2AC:
    reader_AT2AC = csv.reader(file_AT2AC)
    for row in reader_AT2AC:
    # row is a list:
    #['(AT1)(AT2)', 'AClabel', 'sent'] AClabel is 1,2,3,4,5
        ATraw=row[0].strip('()').split(')(')
        samesentAT=[ele for ele in ATraw]
        AT2AC_AT.append(samesentAT)
        AT2AC_AC.append(Label_AC[row[1]])
        AT2AC_Num+=1
file_AT2AC.close()

#I-1-2: get dictionary 
AT2AC_Dict={'food':[], 'service':[], 'price':[], 'ambience':[],'anecdotes/miscellaneous':[] }
for i in range(AT2AC_Num):
    AT2AC_Dict[AT2AC_AC[i]]+=AT2AC_AT[i]
    #AT2AC_Dict is a dictionray with 5 AC lists, in which contain the corresponding AT.

                    ########################################################
                    ############  II. Get outcomes of subtask4  ############
                    ########################################################

FinalDict_RST={}
cnt=0
for ID in range(1, task2SentsNum+1):
    FinalDict_RST[str(ID)]={'sentence':Task2All_dict[str(ID)]['sentence'], 'AspectTerm':[], 'AspectCategroy':[]}
    if len(Task2All_dict[str(ID)]['AspectTerm'][0][0])!=0:
        FinalDict_RST[str(ID)]['AspectTerm']=Task2All_dict[str(ID)]['AspectTerm']
        ATP_SameSent=[pair[1] for pair in Task2All_dict[str(ID)]['AspectTerm']]
        if len(list(set(ATP_SameSent)))==1:
            ACP=ATP_SameSent[0]
            AC_P_SameSent=[tuple((ac, ACP)) for ac in Task2All_dict[str(ID)]['AspectCategroy']]
            FinalDict_RST[str(ID)]['AspectCategroy']=AC_P_SameSent
        elif len(list(set(ATP_SameSent)))>1:
            AC_P_SameSent=[]
            for at in Task2All_dict[str(ID)]['AspectTerm']:
                currentAC=''
                if at[0] in AT2AC_Dict['food']:
                    currentAC='food'
                elif at[0] in AT2AC_Dict['service']:
                    currentAC='service'
                elif at[0] in AT2AC_Dict['price']:
                    currentAC='price'
                elif at[0] in AT2AC_Dict['ambience']:
                    currentAC='ambience'
                else :
                    currentAC='anecdotes/miscellaneous'
                currentACP=at[1]
                AC_P_SameSent.append(tuple((currentAC, currentACP)))
            AC_P_SameSent=list(set(AC_P_SameSent))
            FinalDict_RST[str(ID)]['AspectCategroy']=AC_P_SameSent
    else:
        blob = TextBlob(Task2All_dict[str(ID)]['sentence'])
        tempACP=blob.sentiment.polarity
        AC_P_SameSent=[]
        if tempACP>0:
            for ac in Task2All_dict[str(ID)]['AspectCategroy']:
                AC_P_SameSent.append(tuple((ac, 'positive')))
        elif tempACP<0:
            for ac in Task2All_dict[str(ID)]['AspectCategroy']:
                AC_P_SameSent.append(tuple((ac, 'negatigve')))
        else:
            for ac in Task2All_dict[str(ID)]['AspectCategroy']:
                AC_P_SameSent.append(tuple((ac, 'neutral')))
        FinalDict_RST[str(ID)]['AspectCategroy']=AC_P_SameSent
    FinalDict_RST[str(ID)]['AspectTerm']=list(set(FinalDict_RST[str(ID)]['AspectTerm']))
    FinalDict_RST[str(ID)]['AspectCategroy']=list(set(FinalDict_RST[str(ID)]['AspectCategroy']))
                    
                    ###############################################
                    ############  III. Write into CSV  ############
                    ###############################################              
AT_Write=[]
AC_Write=[]
Sent_Write=[]
for ID in range(1,task2SentsNum+1):
    Sent_Write.append(FinalDict_RST[str(ID)]['sentence'])
    sameSentAT=''
    sameSentAC=''
    for ele1 in FinalDict_RST[str(ID)]['AspectTerm']:
        sameSentAT+='('+ele1[0]+'#'+ele1[1]+')'
    for ele2 in FinalDict_RST[str(ID)]['AspectCategroy']:
        sameSentAC+='('+ele2[0]+'#'+ele2[1]+')'
    AT_Write.append(sameSentAT)
    AC_Write.append(sameSentAC)

with open( '/Users/xizhaohan/Desktop/498estimate/Subtask4_FinalOutcome_trial_restaurant.csv', 'w') as file:
    writer = csv.writer(file)
    for col1,col2,col3 in zip(AT_Write,AC_Write,Sent_Write):
        writer.writerow([str(col1),str(col2),str(col3)])
file.close()

stop=time()
print('Total time is:', stop-start,'seconds')


Total time is: 0.01382303237915039 seconds


In [ ]:
#Then we can count the correct estimate by eyes 
#between Trial_restaurant_origianlContent.csv and Subtask4_FinalOutcome_trial_restaurant.csv
#after all, there are only 100 sentences